In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
fraud = pd.read_csv(r"C:\Users\lenovo\Desktop\DATA SCIENCE\Fraud_Check.csv")

In [3]:
data = fraud.head()

In [4]:
fraud["income"]="<=30000"
fraud.loc[fraud["Taxable.Income"]>=30000,"income"]="Good"
fraud.loc[fraud["Taxable.Income"]<=30000,"income"]="Risky"

In [5]:
fraud["income"].unique()
fraud["income"].value_counts()

Good     476
Risky    124
Name: income, dtype: int64

In [6]:
fraud = fraud.drop(['Taxable.Income'],axis=1)
fraud.rename(columns={"Marital.Status":"marital","City.Population":"population","Work.Experience":"workexp"},inplace=True)
fraud.isnull().sum()

Undergrad     0
marital       0
population    0
workexp       0
Urban         0
income        0
dtype: int64

In [7]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for column_name in fraud.columns:
    if fraud[column_name].dtype == object:
        fraud[column_name] = le.fit_transform(fraud[column_name])
    else:
        pass

In [8]:
features = fraud.iloc[:,0:5]
labels = pd.DataFrame(fraud.iloc[:,5])
fraud["income"].value_counts()

0    476
1    124
Name: income, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features,labels, test_size=0.3, stratify=labels)

In [10]:
print(y_train["income"].value_counts())
print(y_test["income"].value_counts())

0    333
1     87
Name: income, dtype: int64
0    143
1     37
Name: income, dtype: int64


In [11]:
colnames = list(fraud.columns)
predictors = colnames[:5]
target = colnames[5]

In [12]:
fraud.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   Undergrad   600 non-null    int32
 1   marital     600 non-null    int32
 2   population  600 non-null    int64
 3   workexp     600 non-null    int64
 4   Urban       600 non-null    int32
 5   income      600 non-null    int32
dtypes: int32(4), int64(2)
memory usage: 18.9 KB


In [13]:
from sklearn.tree import DecisionTreeClassifier as DT

In [14]:
model = DT(criterion = 'entropy')
model.fit(x_train,y_train)

DecisionTreeClassifier(criterion='entropy')

In [15]:
from sklearn.metrics import accuracy_score

In [16]:
pred_train = pd.DataFrame(model.predict(x_train))

In [17]:
acc_train = accuracy_score(y_train,pred_train) 

In [18]:
from sklearn.metrics import confusion_matrix

In [19]:
cm = pd.DataFrame(confusion_matrix(y_train,pred_train))

In [20]:
pred_test = pd.DataFrame(model.predict(x_test))

In [21]:
acc_test = accuracy_score(y_test,pred_test)

In [22]:
cm_test = confusion_matrix(y_test,pred_test)

In [24]:
from sklearn.tree import export_graphviz

In [25]:
import six
import sys
sys.modules['sklearn.externals.six'] = six

In [26]:
from sklearn.externals.six import StringIO

In [27]:
dot_data = StringIO()
export_graphviz(model, out_file = dot_data ,filled = True,rounded =True,feature_names = predictors,class_names = target, impurity = False)

In [28]:
import pydotplus

In [29]:
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())

In [30]:
graph